In [7]:
import pandas as pd
import numpy as np
from math import sqrt
from math import pi
from math import exp
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [8]:
headers = ['Pregnancy Count', 'Glucose', 'Blood Pressure', 'Skin Thickness', 'Insulin', 'BMI', 'Diabetes Pedigree Function', 'Age', 'Outcome']
df=pd.read_csv("/content/Pima.csv",names=headers)
df.head()

,Pregnancy Count,Glucose,Blood Pressure,Skin Thickness,Insulin,BMI,Diabetes Pedigree Function,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
y = df['Outcome']
x = df.drop(['Outcome'],axis=1)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=42)


In [11]:
x_train.describe()


,Pregnancy Count,Glucose,Blood Pressure,Skin Thickness,Insulin,BMI,Diabetes Pedigree Function,Age
count,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000
mean,3.831597,120.767361,69.170139,20.723958,77.899306,32.064583,0.480200,33.536458
std,3.312864,31.771380,18.699887,15.877307,107.415003,7.861032,0.333188,11.878752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.600000,0.245750,24.000000
50%,3.000000,116.500000,72.000000,23.000000,40.000000,32.400000,0.384000,30.000000
75%,6.000000,141.000000,80.000000,32.000000,129.250000,36.525000,0.646250,41.000000
max,17.000000,199.000000,122.000000,99.000000,744.000000,67.100000,2.329000,81.000000


In [12]:
train_mean_pos = x_train[y_train==1].mean()
train_std_pos = x_train[y_train==1].std()


In [13]:
train_mean_neg = x_train[y_train==0].mean()
train_std_neg = x_train[y_train==0].std()

## **NAIVE BAIYES**

In [14]:
model = GaussianNB()
model.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [15]:
confusion_matrix(y_test,model.predict(x_test))


array([[96, 29],
       [26, 41]])

In [16]:
accuracy_score(y_test.tolist(),model.predict(x_test))

0.7135416666666666

## **NAIVE BAYES FROM SCRATCH**

In [17]:
def cond_probability(x, mean, std):
    exponent = exp(-((x - mean)**2/(2*std**2)))
    return (1 / (sqrt(2*pi)*std)) * exponent

In [18]:
def predict(row):
    prob_pos = len(x_train[y_train==1]) / len(x_train)
    
    for i in range(0,len(row)):
        prob_pos = prob_pos * cond_probability(row[i],train_mean_pos[i],train_std_pos[i])
        
    prob_neg = len(x_train[y_train==0]) / len(x_train)
    
    for i in range(0,len(row)):
        prob_neg = prob_neg * cond_probability(row[i],train_mean_neg[i],train_std_neg[i])    
    
    return [prob_pos,prob_neg]

In [19]:
predictions_raw = []

for row in x_test.values.tolist():
    predictions_raw.append(predict(row))

In [20]:
predictions_raw[0]


[1.6299028206157718e-14, 1.0044068228290291e-14]

In [21]:

predictions = []
for row in predictions_raw:
    if(row[0]>row[1]):
        predictions.append(1)
    else:
        predictions.append(0)

In [22]:
accuracy_score(y_test.tolist(),predictions)*100


71.35416666666666

In [23]:
confusion_matrix(y_test.tolist(),predictions)


array([[96, 29],
       [26, 41]])